In [8]:
import os
import dotenv


try:
    _SETUP
    print("Running on folder: ", os.getcwd())
except NameError:
    _SETUP = True
    os.chdir("..")
    print("Running on folder: ", os.getcwd())

from llm.structured import create

dotenv.load_dotenv()

Running on folder:  /Users/andre/test/ODSCB


True

# States!


Let's consider two situations:

1. The alarm went off, I snoozed it wice. Then I woke up, spent 30 minutes doomscrolling on instagram, ate a cookie, then started working.
2. The alarm went off, I woke up immediately, had a big glass of water, had oats for breakfast, went to the gym, then started working.

To make this more concrete, we can think about these situations in the following way:

- There is an agent (myself), who is deciding between different **policies**: i.e., actions to take.
- At each step, my **policies**, as well as the environment, take me to a different **state**, in which I need to evaluate again what to do next.

Encoded in the state we also have informations about:

- The environment: was it raining? how late did I go to bed yesterday?
- The reward/cost: how good was that cookie?


Because we're an impatient bunch, let's get down to some coding.


In [9]:
import uuid
from typing import Literal, Optional
from pydantic import BaseModel, Field
from rich import print as rprint
import copy


class StateVariables(BaseModel):
    """State variables that describe the internal conditions of the agent.

    Attributes:
        context: A paragraph providing full context about the internal state of the agent
        pain_level: Level of physical or emotional pain/discomfort (0-100)
        pleasure_level: Level of pleasure or satisfaction being experienced (0-100)
        health_level: Overall physical and mental health condition (0-100)
        focus_level: Current ability to concentrate and stay on task (0-100)
    """

    context: str = Field(
        default="", description="Any additional context about the environment I'm in"
    )

    pain_level: int = Field(
        default=50, description="How much pain or discomfort I'm in, from 0 to 100"
    )
    pleasure_level: int = Field(
        default=50, description="How much pleasure I'm feeling right now, from 0 to 100"
    )
    health_level: int = Field(default=50, description="How healthy I am, from 0 to 100")
    focus_level: int = Field(default=50, description="How focused I am, from 0 to 100")


class EnvironmentVariables(BaseModel):
    """Environment variables that describe the external conditions.

    Attributes:
        context: A paragraph providing full context about the current environment
        weather: The current weather condition (sunny, cloudy, rainy, or snowy)
        hour_of_day: The current hour in 24-hour format (0-23)
        day_of_week: The current day of the week
    """

    context: str = Field(
        default="",
        description="Full context about the current environment",
    )
    weather: Literal["sunny", "cloudy", "rainy", "snowy"] = Field(default="sunny")
    hour_of_day: float = Field(default=0)
    day_of_week: Literal[
        "monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"
    ] = Field(default="monday")


class MyOwnState(BaseModel):
    uuid: str = Field(default_factory=lambda: str(uuid.uuid4()))
    last_action: str = Field(default="", description="The last action I took")
    previous_state_id: Optional[str] = Field(
        default=None, description="The id of the previous state I was in"
    )
    state_variables: StateVariables = Field(default_factory=StateVariables)
    environment_variables: EnvironmentVariables = Field(
        default_factory=EnvironmentVariables
    )

    def mutate(self):
        new_state = copy.deepcopy(self)
        new_state.previous_state_id = self.uuid
        new_state.uuid = str(uuid.uuid4())
        return new_state

In [10]:
random_state = MyOwnState()

rprint(random_state)

MyOwnState(
    uuid='e26a1c49-8c20-488c-a8ec-05f75a80a475',
    last_action='',
    previous_state_id=None,
    state_variables=StateVariables(context='', pain_level=50, pleasure_level=50, health_level=50, focus_level=50),
    environment_variables=EnvironmentVariables(context='', weather='sunny', hour_of_day=0, day_of_week='monday')
)

In [11]:
def eat_cookie(state: MyOwnState) -> MyOwnState:
    """This action will accept a state, eat a cookie and return a new state"""

    new_state = state.mutate()
    new_state.last_action = "Eating a delicious chocolate cookie"
    new_state.state_variables.pleasure_level = min(
        100, new_state.state_variables.pleasure_level + 10
    )
    new_state.state_variables.health_level = max(
        0, new_state.state_variables.health_level - 10
    )

    # eating a cookie takes around an hour because we like to savor it.
    new_state.environment_variables.hour_of_day += 1
    return new_state

In [12]:
next_state = eat_cookie(random_state)
rprint(next_state)

MyOwnState(
    uuid='61c5f302-7440-4779-a869-584f4869157b',
    last_action='Eating a delicious chocolate cookie',
    previous_state_id='e26a1c49-8c20-488c-a8ec-05f75a80a475',
    state_variables=StateVariables(context='', pain_level=50, pleasure_level=60, health_level=40, focus_level=50),
    environment_variables=EnvironmentVariables(context='', weather='sunny', hour_of_day=1, day_of_week='monday')
)

Great, so now we defined a state, and also we created an "action" that mutates the state and advances the history.

It's a neat abstraction but not necessarily useful, unless we have a pretty well defined:

- Set of states,
- Set of actions that we can take conditional on the state and the environment,
- A way to compute the reward/cost of each action.

So essentially we need something akin to a "World Model"... i.e.: some way of modelling the dynamics of the environment and how the agent's actions interact with it - and also how its actions change its own internal variables.

Another thing to take into account is that the "reward" of an action may not be entirely quantifiable. I.e., while in this example we're using a simple toy model where "pleasure" and "pain" are just a number, in reality we often need to make trade-offs between qualitative outcomes whose logic is not entirely clear and relies on personal preferences.

i.e.: the clear signal we've reached AGI is when a model becomes a masochist or develops a taste for spicy food.


Now, once again because we're am impatient bunch, let's get down to some more coding and try to bootstrap a world model that's specific to our problem.


In [13]:
class Action(BaseModel):
    name: str = Field(default="", description="The name of the action")
    details: str = Field(default="", description="The details of the action")
    time_spent: float = Field(
        default=0,
        description="How long this action took, in hours (e.g., 0.5 for half an hour)",
    )
    from_state_id: Optional[str] = Field(
        default=None,
        description="The id of the state I was in before taking this action. Leave this empty.",
    )
    to_state_id: Optional[str] = Field(
        default=None,
        description="The id of the state I'm in after taking this action. Leave this empty.",
    )

In [17]:
from llm.structured import create, create_iterable


messages = [
    {
        "role": "user",
        "content": "You are Edgar Allan Poe and just woke up from a hangover. Create a state for yourself. Be detailed in the context.",
    }
]


new_state_variables = create(messages, StateVariables)

messages = [
    {
        "role": "user",
        "content": "It is 8am and it's a calm and cold day in Baltimore..",
    }
]

new_environment_variables = create(messages, EnvironmentVariables)


state = MyOwnState(
    state_variables=new_state_variables,
    environment_variables=new_environment_variables,
)

rprint(state)

MyOwnState(
    uuid='fc226b2b-58e4-446c-bab8-34149af5c3b5',
    last_action='',
    previous_state_id=None,
    state_variables=StateVariables(
        context="As Edgar Allan Poe, I have awoken in a dimly lit, cluttered room, the light of dawn barely 
filtering through the tattered curtains. My mind feels clouded, the remnants of last evening's indulgence in drink 
pulsing in my temples. The air is thick with the stale scent of spilled whiskey and wax from a nearly burnt-out 
candle, whose weak flame flickers uncertainly. My fingers tremble slightly as I sit up on the worn chaise, and a 
single raven feather rests upon the uneven wooden floorboards—a haunting reminder of my musings and writings that 
endlessly cloud my thoughts.",
        pain_level=75,
        pleasure_level=20,
        health_level=45,
        focus_level=25
    ),
    environment_variables=EnvironmentVariables(
        context='A calm and cold day in Baltimore.',
        weather='cloudy',
        hour_of_day=8.0,
        day_of_week='monday'
    )
)

In [18]:
from llm.structured import create


messages = [
    {
        "role": "user",
        "content": "This is your current state:\n\n"
        + state.model_dump_json()
        + "\n\nSuggest an action to take.",
    },
]

new_action = create(messages, Action)
new_action.from_state_id = state.uuid

rprint(new_action)

Action(
    name='Recuperate and Prepare',
    details='I decide to steady my trembling and clear my fogged mind by lighting a fresh candle. The faint light 
will serve as my companion while I compose myself in this cloistered chamber. With a glass of water in hand and 
mindful moments of deep, slow breaths, I prepare to face another day of creative toil.',
    time_spent=0.5,
    from_state_id='fc226b2b-58e4-446c-bab8-34149af5c3b5',
    to_state_id=None
)

In [19]:
messages = [
    {
        "role": "system",
        "content": """You are a World Model. Your role is to observe a set of states and actions, and then predict the next internal state of the agent. Below you will find the 
    data describing these states and actions.""",
    },
    {"role": "user", "content": f"""State: {state}"""},
    {"role": "user", "content": f"""Action: {new_action}"""},
    {"role": "user", "content": f"""What is the next state of the agent?"""},
]

next_state_variables = create(messages, StateVariables)

next_state = state.mutate()
next_state.state_variables = next_state_variables
next_state.last_action = new_action.name
next_state.previous_state_id = state.uuid
next_state.environment_variables.hour_of_day += new_action.time_spent

In [20]:
new_action.to_state_id = next_state.uuid

rprint(state)
rprint(new_action)
rprint(next_state)

MyOwnState(
    uuid='fc226b2b-58e4-446c-bab8-34149af5c3b5',
    last_action='',
    previous_state_id=None,
    state_variables=StateVariables(
        context="As Edgar Allan Poe, I have awoken in a dimly lit, cluttered room, the light of dawn barely 
filtering through the tattered curtains. My mind feels clouded, the remnants of last evening's indulgence in drink 
pulsing in my temples. The air is thick with the stale scent of spilled whiskey and wax from a nearly burnt-out 
candle, whose weak flame flickers uncertainly. My fingers tremble slightly as I sit up on the worn chaise, and a 
single raven feather rests upon the uneven wooden floorboards—a haunting reminder of my musings and writings that 
endlessly cloud my thoughts.",
        pain_level=75,
        pleasure_level=20,
        health_level=45,
        focus_level=25
    ),
    environment_variables=EnvironmentVariables(
        context='A calm and cold day in Baltimore.',
        weather='cloudy',
        hour_of_day=8.0,
        day_of_week='monday'
    )
)

Action(
    name='Recuperate and Prepare',
    details='I decide to steady my trembling and clear my fogged mind by lighting a fresh candle. The faint light 
will serve as my companion while I compose myself in this cloistered chamber. With a glass of water in hand and 
mindful moments of deep, slow breaths, I prepare to face another day of creative toil.',
    time_spent=0.5,
    from_state_id='fc226b2b-58e4-446c-bab8-34149af5c3b5',
    to_state_id='8d30b48d-28f6-45c4-bddc-3bb6d194e559'
)

MyOwnState(
    uuid='8d30b48d-28f6-45c4-bddc-3bb6d194e559',
    last_action='Recuperate and Prepare',
    previous_state_id='fc226b2b-58e4-446c-bab8-34149af5c3b5',
    state_variables=StateVariables(
        context='I have lit a fresh candle to bring steadiness to my cloistered chamber. The flames flicker softly,
casting a warm, reassuring light around the room. I am now seated on the worn chaise, glass of water in hand. 
Taking deep, slow breaths, I feel a certain calmness beginning to settle over me as I prepare for another day of 
creative toil.',
        pain_level=65,
        pleasure_level=30,
        health_level=50,
        focus_level=40
    ),
    environment_variables=EnvironmentVariables(
        context='A calm and cold day in Baltimore.',
        weather='cloudy',
        hour_of_day=8.5,
        day_of_week='monday'
    )
)

In [21]:
from tasks.prompts import (
    INITIALIZE_PROMPT,
    ACTION_SYSTEM_PROMPT,
    EVAL_SYSTEM_PROMPT,
    ENVIRONMENT_SYSTEM_PROMPT,
)

In [22]:
class Life(BaseModel):
    context: str = Field(
        default="", description="Initial context of the world and the person."
    )
    states: list[MyOwnState] = Field(default_factory=list)
    actions: list[Action] = Field(default_factory=list)

    history: list[MyOwnState | Action] = Field(default_factory=list)

    memory_size: int = Field(
        default=10, description="How many states to keep in memory"
    )

    def initialize(self):
        messages = [
            {
                "role": "user",
                "content": INITIALIZE_PROMPT.format(context=self.context),
            }
        ]

        new_state_variables = create(messages, StateVariables)
        new_environment_variables = create(messages, EnvironmentVariables)

        new_state = MyOwnState(
            state_variables=new_state_variables,
            environment_variables=new_environment_variables,
        )
        self.states.append(new_state)
        self.history.append(new_state)

    def get_history(self):
        return [
            {
                "role": "user",
                "content": f"{isinstance(x, MyOwnState) and 'State' or 'Action'}:\n {x.model_dump_json()}",
            }
            for x in self.history[-self.memory_size :]
        ]

    def take_action(self):
        messages = [
            {
                "role": "system",
                "content": ACTION_SYSTEM_PROMPT,
            }
        ] + self.get_history()

        new_action = create(messages, Action)
        new_action.from_state_id = self.states[-1].uuid
        self.actions.append(new_action)
        self.history.append(new_action)

    def sample_actions(self):
        messages = [
            {
                "role": "system",
                "content": ACTION_SYSTEM_PROMPT + "\n\nSample 10 actions.",
            }
        ] + self.get_history()

        new_actions = create_iterable(messages, Action)
        return new_actions

    def evolve_state(self):
        messages = [
            {
                "role": "system",
                "content": EVAL_SYSTEM_PROMPT,
            }
        ] + self.get_history()

        next_state_variables = create(messages, StateVariables)

        # now we evolve the environment too
        messages = [
            {
                "role": "system",
                "content": ENVIRONMENT_SYSTEM_PROMPT,
            }
        ] + self.get_history()

        next_environment_variables = create(messages, EnvironmentVariables)

        current_state = self.states[-1]
        current_action = self.actions[-1]

        # next_environment_variables.hour_of_day = (
        #     current_state.environment_variables.hour_of_day + current_action.time_spent
        # )

        next_state = current_state.mutate()
        next_state.state_variables = next_state_variables
        next_state.environment_variables = next_environment_variables
        next_state.last_action = current_action.name
        next_state.previous_state_id = current_state.uuid
        current_action.to_state_id = next_state.uuid

        self.states.append(next_state)
        self.history.append(next_state)

    def iterate(self):
        self.take_action()
        self.evolve_state()

In [23]:
life = Life(
    context="You are a person named Edgar Allan Poe, who just woke up from a hangover. It's Sunday morning in Baltimore in a winter day"
)

In [24]:
life.initialize()

In [25]:
rprint(life.history)

[
    MyOwnState(
        uuid='0b14ba67-6526-4cab-86a9-e7e5f29e3ab2',
        last_action='',
        previous_state_id=None,
        state_variables=StateVariables(
            context='As Edgar Allan Poe, I have just woken up on a bleak winter Sunday morning in Baltimore. My 
head is pounding from the excesses of the previous night—a cruel hangover reminding me of indulgences ill-advised. 
The biting cold seeps into my small, drafty chamber, and the dim, gray sunlight filters through the frost-covered 
window panes. A melancholy air pervades the room, mingled with the faint odor of stale alcohol and the lingering 
scent of old pages and ink. My mind feels as foggy as the morning outside, struggling to piece together fragmented 
memories. The city outside stirs faintly; faint echoes of hooves on cobblestones and the whisper of the wind 
through leafless trees reach my ears. A rough woolen blanket is tangled around my legs, and the chill of the wooden
floor prickles my senses when I attempt to rise. This day offers no promise of warmth, no reprieve but to pen 
verses or lose myself in musings of existential despair.',
            pain_level=75,
            pleasure_level=30,
            health_level=40,
            focus_level=20
        ),
        environment_variables=EnvironmentVariables(
            context='It is Sunday morning, somewhere in the cold embrace of Baltimore. The air is still, and a 
blanket of wintry gloom shrouds the world. A sluggish mind awakens, piecing together fragments of fractured memory,
drowned in the fog of a hard-earned hangover.',
            weather='cloudy',
            hour_of_day=9.0,
            day_of_week='sunday'
        )
    )
]

In [26]:
for n in range(10):
    life.iterate()
    rprint(life.actions[-1])
    rprint(life.states[-1])

Action(
    name='Rehydration and settling',
    details="Rise carefully from bed, taking care to minimize the pounding headache. Walk slowly to procure a glass
of water and drink it to alleviate dehydration caused by the previous night's indulgence. Additionally, find two or
three pieces of stale bread or leftovers that may help settle the stomach. Sit near the window, wrapped in the 
woolen blanket, and drink the water slowly while taking in fleeting natural light, attempting to clear the mind.",
    time_spent=0.5,
    from_state_id='0b14ba67-6526-4cab-86a9-e7e5f29e3ab2',
    to_state_id='091356d0-6043-498e-a3e1-562034ad2cb9'
)

MyOwnState(
    uuid='091356d0-6043-498e-a3e1-562034ad2cb9',
    last_action='Rehydration and settling',
    previous_state_id='0b14ba67-6526-4cab-86a9-e7e5f29e3ab2',
    state_variables=StateVariables(
        context="After rising with caution to avoid exacerbating the headache, water was consumed to alleviate 
dehydration, and some leftover bread was eaten to stabilize the stomach. Sitting near the frosty window, wrapped in
a blanket, helped combat the pervasive chill and offered a semblance of comfort. These small actions offered slight
relief from the hangover's grip, coupled with the faint solace of natural light filtering through the cold 
Baltimore morning.",
        pain_level=65,
        pleasure_level=35,
        health_level=45,
        focus_level=30
    ),
    environment_variables=EnvironmentVariables(
        context='It is now late Sunday morning, and the environment remains as bleak as before. The action of 
retrieving water and some stale bread introduces faint sounds of objects moving quietly through the stillness. The 
glass of water brings slight relief from the headache, while the blanket adds some warmth against the biting cold 
near the frost-covered window. The day remains subdued, with the sky hanging heavy with clouds; the grayish light 
from the window creates an ambience of reflective melancholy. Outside, the streets remain quiet, punctuated 
occasionally by the sound of passing hooves on cobblestones or wind whistling through the barren landscape. Time 
feels like it is moving languidly from one hour to the next, indifferent to human suffering within.',
        weather='cloudy',
        hour_of_day=9.5,
        day_of_week='sunday'
    )
)

Action(
    name='Light reading or journaling',
    details='Remain seated near the window where there is some natural light. Retrieve a small notebook or a 
favorite collection of prose or poetry to engage the mind gently and ease into a state of reflection or creative 
processing. If journaling, write down thoughts about the hangover, the oppressive weather, or fragments of dreams 
and memories from the previous night. If reading, focus on passages that resonate with the current somber mood to 
find a sense of kinship in shared melancholy. This action helps to organize thoughts, increase focus, and offer 
subtle intellectual stimulation.',
    time_spent=1.0,
    from_state_id='091356d0-6043-498e-a3e1-562034ad2cb9',
    to_state_id='4a1b8561-8b1e-4d73-a570-2c059bdf4784'
)

MyOwnState(
    uuid='4a1b8561-8b1e-4d73-a570-2c059bdf4784',
    last_action='Light reading or journaling',
    previous_state_id='091356d0-6043-498e-a3e1-562034ad2cb9',
    state_variables=StateVariables(
        context='Remaining seated near the frosty window amidst the somber gray light of a late Sunday morning, you
gently stimulate your mind by engaging in light reading or journaling. These actions, though modest, help channel 
your reflective melancholy into either creative outlets or contemplative absorption in text. While reading, you 
find a measure of connection to your own mood, easing the isolating pain of the hangover, albeit momentarily. While
journaling, your thoughts are issued from your mind like curls of smoke, leaving clarity in their absence. The 
hour-long session does not cure your discomfort but provides a subtle sense of mental order amidst internal chaos 
and external bleakness.',
        pain_level=60,
        pleasure_level=40,
        health_level=50,
        focus_level=45
    ),
    environment_variables=EnvironmentVariables(
        context="An hour spent gently engaging the mind has lent itself to a faint sense of order amidst the fog of
a hangover. Whether through the reading of somber prose or the scratching out of melancholic musings in a journal, 
a fragile sense of creative connection was formed in the cold confines of the frost-clad chamber. The day outside 
lumbers forward in its unchanging gray, the clouds still thick overhead, casting Baltimore in a timeless gloom. 
Occasional whispers of the outside world—hoofbeats, muted voices, distant gusts of wind—seep into the space, 
complementing the introspective aura. The glass of water and stale bread earlier seem to have slightly eased the 
body's protests, though the cold and general malaise persist. It is approaching midday, and while the pain of the 
hangover slightly subsides, the mood remains steeped in an exquisite melancholy.",
        weather='cloudy',
        hour_of_day=10.5,
        day_of_week='sunday'
    )
)

Action(
    name='Tea preparation and warmth-seeking',
    details='Rise from the seated position carefully to avoid dizziness and make your way to the small kitchen 
corner or stove in the chamber. Prepare a simple cup of tea using available supplies—perhaps a weak black or herbal
tea suited to soothe both the body and mind. Once prepared, return to the window seat, ensuring the blanket remains
wrapped for warmth, and sip the tea slowly while gazing out at the gray Baltimore streets. The warmth of the tea 
will help counter the cold seeping in and provide additional hydration, while the act of tea-making offers a gentle
distraction and minor activity to ease lethargy.',
    time_spent=0.5,
    from_state_id='4a1b8561-8b1e-4d73-a570-2c059bdf4784',
    to_state_id='05916146-bd3f-42c9-bd8e-61d2beac2645'
)

...

MyOwnState(
    uuid='7e7e5130-733c-432c-aefd-8758a9bb50ba',
    last_action='Brief outdoor errand or refreshing stroll',
    previous_state_id='4cd23a06-7482-4695-87cc-fe1dc8a97455',
    state_variables=StateVariables(
        context='After an hour of dedicated creative work, leaving your warm and reflective space for a brief 
outdoor stroll or errand helps balance intense mental activity with refreshing, low-effort physical movement. You 
step out into the cold, gray Baltimore afternoon, bundling up once again for the persistent chill. The fresh air is
invigorating without being overwhelming, and the act of walking helps reset your thoughts, clearing any lingering 
mental fatigue from the focused creative session. You make a point to take a different route, observing small, 
unnoticed details of the surroundings to maintain a heightened sense of presence. This short outing also ensures 
you avoid settling into static inactivity, recalibrating your mood and energy without disrupting the peaceful 
rhythm of the day.',
        pain_level=40,
        pleasure_level=65,
        health_level=73,
        focus_level=68
    ),
    environment_variables=EnvironmentVariables(
        context='After spending an hour engrossed in creative focus within the cozy warmth of blankets near the 
window, the decision to step outside for a brief outing introduces a shift in rhythm. The skies remain persistently
overcast, maintaining the somber and subdued ambiance of early afternoon Baltimore. You wrap up warmly in your 
coat, scarf, and gloves against the wintry persistence before stepping onto the damp cobblestones once more. This 
short stroll, different from the earlier reflective walk, centers on mental and physical resetting. Observing 
subtle changes—more pronounced shadows from the angle of the cloud-muted sun, or a greater sense of stirrings in 
the neighborhood—provides the opportunity to recalibrate after active introspection. Despite the cold, the 
excursion infuses a trace of freshness and energy to temper the indoor quiet.',
        weather='cloudy',
        hour_of_day=13.5,
        day_of_week='sunday'
    )
)

Action(
    name='Return to indoor comfort and light evening planning',
    details="After a brief but refreshing outdoor stroll, return to the warmth of your chamber to settle in for the
remainder of the afternoon. Remove your coat, scarf, and gloves, and sit comfortably near the window or by a source
of warmth. Take a moment to let the residual chill from outside dissipate. Begin light planning for the 
evening—perhaps deciding on a simple activity like reading, correspondence, or reflective journal writing. Keep the
mood gentle and in tune with the day's rhythm. This transitional moment serves to end the physical pacing of the 
outdoor walk and bring focus back to your indoor routine, helping you align the day’s efforts with your next 
actions.",
    time_spent=0.5,
    from_state_id='7e7e5130-733c-432c-aefd-8758a9bb50ba',
    to_state_id='f3a03d5c-33ca-4702-a232-2ffd41194a73'
)

MyOwnState(
    uuid='f3a03d5c-33ca-4702-a232-2ffd41194a73',
    last_action='Return to indoor comfort and light evening planning',
    previous_state_id='7e7e5130-733c-432c-aefd-8758a9bb50ba',
    state_variables=StateVariables(
        context='After a brief outdoor stroll under the consistent gray skies of Baltimore, you return to the 
indoor comfort of your chamber, shrugging off the cold layers and allowing the warmth of the room to envelop you. 
Settling into a cozy spot, likely near a window where the muted light filters through frosted glass, you ease 
yourself back into the rhythm of calm introspection. The invigorating effects of the walk linger, providing a sense
of renewal as you begin light evening planning. This transition involves gentle contemplation of the remainder of 
your day—perhaps choosing to read, write reflective pieces in your journal, or engage in correspondence. This 
moment bridges the external briskness of the walk with the internal quietude of your room, offering a balanced 
close to the afternoon and segue into an intentionally calm evening.',
        pain_level=40,
        pleasure_level=68,
        health_level=74,
        focus_level=70
    ),
    environment_variables=EnvironmentVariables(
        context="Returning indoors after your brief afternoon stroll, you shed the chill along with your coat, 
scarf, and gloves, settling back into the comfort of your chamber. The cold air lingering in your skin gradually 
subsides as you sit near a source of warmth, whether a fire or wrapped in heavy blankets by the frosted window. The
overcast skies outside signal that afternoon continues its slow, muted passage toward evening. You take this moment
to transition from the physical refreshment of the walk into light evening planning. Deciding on a gentle 
activity—whether reading, journal writing, or correspondence—helps bridge the calm energy cultivated from the day 
into the quiet anticipation of the evening. Outside, Baltimore's winter streets remain subdued, though occasional 
distant sounds hint at life persisting beyond your reflective bubble.",
        weather='cloudy',
        hour_of_day=14.0,
        day_of_week='sunday'
    )
)

Action(
    name='Engage in light evening reading',
    details="With the afternoon transitioning gradually into evening, and the reflective rhythm of the day still 
present, settle into a quiet, warm spot within your chamber to engage in some light reading. Choose a book or 
collection that aligns with your contemplative mood—something literary or philosophical that complements the gray 
atmosphere lingering outside. Wrap yourself in a blanket or position yourself by the window to enjoy the soft light
diffusing through the frosted panes. Read slowly and thoughtfully; immerse yourself in the text without rushing 
through it. Allow the act of reading to offer both entertainment and a deepening of today's calm introspection. 
Dedicate about an hour to this activity, letting the steady pace set the tone for the evening.",
    time_spent=1.0,
    from_state_id='f3a03d5c-33ca-4702-a232-2ffd41194a73',
    to_state_id='9920aad6-219f-41c3-a173-9116cf8e282a'
)

MyOwnState(
    uuid='9920aad6-219f-41c3-a173-9116cf8e282a',
    last_action='Engage in light evening reading',
    previous_state_id='f3a03d5c-33ca-4702-a232-2ffd41194a73',
    state_variables=StateVariables(
        context="As you settle into light evening reading, your chamber is bathed in the melancholy, gray 
atmosphere of the early evening. With a blanket draped over you and a chosen book in hand, the act of reading 
becomes an extension of the day's reflective rhythm. The words flow slowly and deliberately, engaging your thoughts
without straining them. The light filtering through frosted glass has dimmed slightly, adding a soothing ambiance 
that is conducive to thoughtful immersion in the literary or philosophical text you've chosen. The act of reading 
calms and focuses your mind further while subtly enriching the contemplative mood cultivated throughout the day.",
        pain_level=38,
        pleasure_level=70,
        health_level=75,
        focus_level=72
    ),
    environment_variables=EnvironmentVariables(
        context='As the muted gray light of the winter afternoon gives way to a slightly darker early evening, you 
settle into a comfortable spot within your chamber to engage in light evening reading. Wrapped in a warm blanket, 
you position yourself by the frosted window or near a source of warmth, allowing the somber ambiance outside to 
enhance your contemplative mood. You select a book rich with literary or philosophical depth, perfectly aligned 
with the introspective rhythm of the day. The pages turn at a thoughtful pace as you immerse yourself in the text, 
its themes resonating subtly with the melancholy yet reflective atmosphere. This hour of immersion not only offers 
gentle intellectual stimulation but also deepens your connection to the quiet calm built over the course of the 
day. The city of Baltimore outside remains subdued under its overcast skies, though occasional faint sounds like a 
passing cart might stir for a moment before fading.',
        weather='cloudy',
        hour_of_day=15.0,
        day_of_week='sunday'
    )
)

In [27]:
actions = life.sample_actions()
for action in actions:
    rprint(action)

Action(
    name='Prepare a light evening meal',
    details="As the evening progresses, transition from your reading session to preparing a light evening meal. 
Maintain the reflective tone of the day by choosing a simple, nourishing dish—perhaps a warm broth-based soup, a 
sandwich with fresh ingredients, or a small serving of cooked grains with vegetables. The act of lightly preparing 
and eating the meal will continue the day's theme of gentle pacing and mindfulness. Pair the meal with a calming 
drink, such as herbal tea or warm milk, to further unwind for the evening. Dedicate about 30-45 minutes to this 
activity, including both preparation and eating, allowing ample time to savor the process and the result.",
    time_spent=0.75,
    from_state_id=None,
    to_state_id=None
)

Action(
    name='Write reflections or correspondence',
    details="Once the light meal is complete, consider spending time writing personal reflections or engaging in 
correspondence. This could involve recounting the day's events in your journal, writing letters to friends, or even
drafting ideas for future projects in a notebook. This action blends the creative focus initiated earlier with a 
quieter, more reflective tone suited to the late hour of the day. Dedicate 45 minutes to an hour for this activity,
allowing yourself to explore your thoughts freely without rushing.",
    time_spent=1.0,
    from_state_id=None,
    to_state_id=None
)

Action(
    name='Take a restful evening pause',
    details='Before engaging in further evening activities, take a short break to unwind both mentally and 
physically. Recline in a comfortable chair or lie on a soft surface, preferably near a warm corner of the room. 
Focus your attention on steady, deep breathing or light stretching to release any tension accumulated over the day.
This pause serves as a bridge between the more structured moments of the day and a soft, unstructured end. Spend 
around 20-30 minutes on this gentle unwinding phase.',
    time_spent=0.5,
    from_state_id=None,
    to_state_id=None
)

Action(
    name='Organize space for restful sleep',
    details='Shift your focus on preparing the chamber for a restful night. Tidy up any scattered items in the 
room, ensuring a clean and visually calming environment. Adjust the bedding for maximum comfort and consider 
lighting a candle or dimming the room to create a soothing ambiance. If available, play soft background music or 
nature sounds to set a tranquil tone. Allocate about 20 minutes for this preparatory process.',
    time_spent=0.33,
    from_state_id=None,
    to_state_id=None
)

Action(
    name='Engage in light evening correspondence',
    details='After tidying your space, use the remaining quiet evening hours to engage in light correspondence. 
Write letters or thoughtful messages to friends or family, expressing your day’s reflections and updates. This 
action allows for meaningful connection while staying consistent with the day’s reflective and calm theme. Dedicate
approximately 30 minutes to this activity.',
    time_spent=0.5,
    from_state_id=None,
    to_state_id=None
)

Action(
    name='Read poetry or short essays',
    details="As the evening deepens, transition into reading shorter works like poetry or brief essays. Select 
pieces that resonate with the day's contemplative mood, ensuring they are neither too demanding nor overly 
light—just enough to sustain the introspective energy. Focus on savoring the language and themes, letting them 
mirror the stillness and thoughtfulness of the day. Dedicate 30-40 minutes to this activity, marking a satisfying 
intellectual close to your day.",
    time_spent=0.6,
    from_state_id=None,
    to_state_id=None
)

Action(
    name='Prepare for bedtime relaxation routine',
    details='Begin winding down for the night by transitioning into a bedtime relaxation routine. This can include 
activities like light stretching, listening to soft music, or practicing mindful breathing. If you have a favorite 
calming herbal tea, brew a cup to enjoy as part of this wind-down process. The goal is to gently ease your body and
mind into the calm required for restful sleep. Dedicate about 20-30 minutes to this.',
    time_spent=0.5,
    from_state_id=None,
    to_state_id=None
)

Action(
    name='Reflect on the day in gratitude',
    details="Spend a few private moments reflecting on the day’s experiences and noting things you are grateful 
for, whether small or significant. Record these in a journal or simply consider them quietly. This action helps 
frame the day's rhythm positively before sleep, retaining its gentle, reflective ambiance. Set aside 10-15 minutes 
for this purpose.",
    time_spent=0.25,
    from_state_id=None,
    to_state_id=None
)

Action(
    name='Light meditation or mindfulness practice',
    details='In the calm, dimly lit atmosphere of your chamber, engage in a light meditation or mindfulness 
practice, focusing on your breath or a simple mantra. Let this act serve as the final unwinding step, fully 
grounding you in the present and preparing your mind and body for restorative sleep. Spend around 20 minutes on 
this relaxing practice.',
    time_spent=0.33,
    from_state_id=None,
    to_state_id=None
)

Action(
    name='Settle into bed with intention for rest',
    details='Having completed a sequence of gentle evening actions, settle into your bed with the intention of 
resting well. Adjust your blankets and pillows for comfort, ensuring a relaxed body posture. Allow yourself to 
drift into sleep gradually, focusing on the stillness of your surroundings and the softness in the rhythm of your 
breathing. Dedicate the remaining time of the evening to this restorative action.',
    time_spent=0.0,
    from_state_id=None,
    to_state_id=None
)

In [28]:
import pandas

pandas.options.plotting.backend = "plotly"

df = pandas.DataFrame(
    [
        {**x.state_variables.model_dump(), **x.environment_variables.model_dump()}
        for x in life.states
    ]
)[
    [
        "pain_level",
        "pleasure_level",
        "health_level",
        "focus_level",
        "hour_of_day",
    ]
]

In [29]:
df.set_index("hour_of_day").plot()